In [2]:
import os
import cv2
import numpy as np
from glob import glob
import random

def get_images(folder):
    """Return a list of all .jpg images in the given folder."""
    return glob(os.path.join(folder, '*.jpg'))

def calculate_batch_avg_and_std_rgb(batch_images):
    """
    Calculate the per-channel average and standard deviation of the images in a batch.
    Returns:
      avg_rgb: (r_avg, g_avg, b_avg)
      std_rgb: (r_std, g_std, b_std) -- with zero std replaced by 1.
    """
    r_values, g_values, b_values = [], [], []
    for img_path in batch_images:
        img = cv2.imread(img_path)
        if img is None:
            print(f"Warning: Could not read image {img_path}")
            continue
        b, g, r = cv2.split(img)
        r_values.append(np.mean(r))
        g_values.append(np.mean(g))
        b_values.append(np.mean(b))
        
    if not r_values or not g_values or not b_values:
        return (np.nan, np.nan, np.nan), (np.nan, np.nan, np.nan)
    
    r_avg = np.mean(r_values)
    g_avg = np.mean(g_values)
    b_avg = np.mean(b_values)
    
    r_std = np.std(r_values)
    g_std = np.std(g_values)
    b_std = np.std(b_values)
    
    # Prevent division by zero
    r_std = r_std if r_std != 0 else 1
    g_std = g_std if g_std != 0 else 1
    b_std = b_std if b_std != 0 else 1
    
    return (r_avg, g_avg, b_avg), (r_std, g_std, b_std)

def adjust_image_color(img, batch_avg_rgb, batch_std_rgb, overall_avg_rgb):
    """
    Normalize the image by:
      - Subtracting the batch average and dividing by the batch standard deviation (per channel)
      - Then adding the overall average so that the images are re-centered.
    """
    b, g, r = cv2.split(img)
    r_adjusted = ((r - batch_avg_rgb[0]) / batch_std_rgb[0])
    g_adjusted = ((g - batch_avg_rgb[1]) / batch_std_rgb[1])
    b_adjusted = ((b - batch_avg_rgb[2]) / batch_std_rgb[2])
    
    adjusted_img = cv2.merge([b_adjusted, g_adjusted, r_adjusted])
    return adjusted_img

def process_batch(batch_images, batch_avg, batch_std, overall_avg_rgb, output_folder, batch_name):
    """
    Process all images in a batch:
      - Normalize the image using the batch stats and overall average.
      - Print a random pixel’s RGB value.
      - Save the normalized image as a compressed .npz file.
    """
    for img_path in batch_images:
        img = cv2.imread(img_path)
        if img is None:
            print(f"Warning: Could not read image {img_path}")
            continue
        
        adjusted_img = adjust_image_color(img, batch_avg, batch_std, overall_avg_rgb)
        
        # Print a random pixel's RGB values for debugging.
        height, width, _ = adjusted_img.shape
        rand_x = random.randint(0, width - 1)
        rand_y = random.randint(0, height - 1)
        random_pixel_rgb = adjusted_img[rand_y, rand_x]
        print(f"Random pixel at ({rand_x}, {rand_y}) in {batch_name} image {os.path.basename(img_path)}: {random_pixel_rgb}")
        
        # Save the normalized image. The filename includes the batch name for clarity.
        npz_filename = f"{batch_name}_" + os.path.basename(img_path).replace('.jpg', '.npz')
        npz_file_path = os.path.join(output_folder, npz_filename)
        np.savez_compressed(npz_file_path, image=adjusted_img)

def main(indoor_folder, outdoor_folder, output_folder):
    # Get images from each folder.
    indoor_images = get_images(indoor_folder)
    outdoor_images = get_images(outdoor_folder)
    
    # Calculate batch stats for indoor and outdoor images.
    indoor_avg, indoor_std = calculate_batch_avg_and_std_rgb(indoor_images)
    outdoor_avg, outdoor_std = calculate_batch_avg_and_std_rgb(outdoor_images)
    
    # Compute overall average RGB as the mean of the two batch averages.
    overall_avg_rgb = np.mean([indoor_avg, outdoor_avg], axis=0)
    
    # Print the statistics.
    print("Indoor batch average (R, G, B):", indoor_avg)
    print("Indoor batch std (R, G, B):", indoor_std)
    print("Outdoor batch average (R, G, B):", outdoor_avg)
    print("Outdoor batch std (R, G, B):", outdoor_std)
    print("Overall average (R, G, B):", overall_avg_rgb)
    
    # Create the output folder if it doesn't exist.
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Process each batch separately.
    process_batch(indoor_images, indoor_avg, indoor_std, overall_avg_rgb, output_folder, 'indoor')
    process_batch(outdoor_images, outdoor_avg, outdoor_std, overall_avg_rgb, output_folder, 'outdoor')

if __name__ == "__main__":
    indoor_folder = r'D:\Download\COCO VAL2017\data\Indoor_cropped'
    outdoor_folder = r'D:\Download\COCO VAL2017\data\Outdoor_cropped'
    output_folder = r'D:\Download\COCO VAL2017\data\Final_Normalized'
    main(indoor_folder, outdoor_folder, output_folder)


Indoor batch average (R, G, B): (np.float64(123.11347936692083), np.float64(107.29016414842106), np.float64(93.43259846599322))
Indoor batch std (R, G, B): (np.float64(38.69158754633211), np.float64(35.92811697372071), np.float64(38.60690110942378))
Outdoor batch average (R, G, B): (np.float64(118.67956002735254), np.float64(117.75912875458538), np.float64(110.49432485335592))
Outdoor batch std (R, G, B): (np.float64(35.58468935425815), np.float64(35.465064970643624), np.float64(41.62518609957815))
Overall average (R, G, B): [120.8965197  112.52464645 101.96346166]
Random pixel at (178, 164) in indoor image 000000000139.jpg: [-2.18698202 -2.01207773 -1.91549337]
Random pixel at (218, 10) in indoor image 000000000632.jpg: [-0.68465994 -1.03791034 -1.42443055]
Random pixel at (136, 14) in indoor image 000000000776.jpg: [-0.5033452   0.54859084  0.9533473 ]
Random pixel at (88, 237) in indoor image 000000000802.jpg: [ 0.53273899  0.13109053 -0.31307786]
Random pixel at (153, 48) in indoor